<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter_sem_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [2]:
!pip install fredapi
!pip install pycountry

from fredapi import Fred
import pycountry
import pandas as pd
from datetime import datetime
import requests

# Inicialize a chave do FRED
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')

# Chave API para taxa de câmbio
exchangeratesapi = '5f749a0aa7b1501d0bec324f453a7336'

# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.4 MB/s eta 0:00:00


In [ ]:
# Importar dados PPP
url = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/eabb3ca9caaca37dc9d9f1ae46ad03f7bc699954/ppp_att.csv'
ppp_data =  pd.read_csv(url, delimiter=',')

## Insira seus dados:

In [3]:
# Dados

## Insira em 'País_Moeda' o o país emissor da moeda utilizada (no caso de uso de moedas diferentes do país de origem), em 'País do Programa' o país de origem do Programa, em 'Ano' o ano da origem dos recursos, em 'Gasto_Local' a quantia dispendida, em 'Programa' o nome do Programa e em 'Descrição' uma breve descrição da função ou destinação dos valores.
## A coluna 'Ano2' serve para os programas que tem custos entre 2 diferentes anos, sendo necessário calcular a média do PPP e da inflação americana. Então, insira no 'Ano2' o último ano do programa, senão preencha com 0.

dados = {
    'País_Moeda': [
             ],  # Usando códigos ISO2 dos países

        'País do Programa': [
             ],  # Usando códigos ISO2 dos países

    'Ano': [
            ],

   'Ano2': [
            ],

    'Gasto_Local': [
                    ],  # Gasto em moeda local

    'Programa': [
                 ],  # Nomes dos programas

    'Descrição': [
                  ]  # Descrições dos programas
}

#Check se todos dados estão com o mesmo número de entradas
print(len(dados['Programa']),
len(dados['Descrição']),
len(dados['País_Moeda']),
len(dados['País do Programa']),
len(dados['Ano']),
len(dados['Ano2']),
len(dados['Gasto_Local']))

df = pd.DataFrame(dados)

0 0 0 0 0 0 0


## Obter taxa de conversão do dólar 2024

In [5]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Função para buscar a taxa de câmbio média do ano de 2024
def get_exchange_rate_2024(currency_code):
    try:
        url = f"https://api.exchangerate-api.com/v4/latest/{currency_code}?access_key={exchangeratesapi}"

        response = requests.get(url)
        data = response.json()
        end_date = datetime.today().strftime('%Y-%m-%d')
        exchange_rate = data['rates'].get('USD')  # Taxa de câmbio para USD
        return exchange_rate
    except Exception as e:
        print(f"Erro ao obter taxa de câmbio para {currency_code}: {e}")
        return None

## Função Conversão

In [6]:
# Função para obter o PPP ajustado de um país em um ano específico a partir do CSV
def get_ppp_adjusted(country_code, year):
    ppp_row = ppp_data[(ppp_data['Country ID'] == country_code) & (ppp_data['Year'] == year)]
    return ppp_row.iloc[0]['PPP']

# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
      if row['Ano'] == 2024:
          currency_code = get_currency_code(row['País_Moeda'])
          if currency_code:
              exchange_rate = get_exchange_rate_2024(currency_code)
              gasto_em_dolar_2024 = row['Gasto_Local'] * exchange_rate
              return gasto_em_dolar_2024
      else:
          # Para demais anos
           ppp_adjusted = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
           gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

      # Ajuste pela inflação dos EUA
      cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    else:
      # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2

      ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
      ppp_ano2 = get_ppp_adjusted(row['País_Moeda'], row['Ano2'])

      # Média dos gastos enre entre Ano1 e Ano2
      gasto_em_dolar_ppp = row['Gasto_Local'] / ((ppp_ano1 + ppp_ano2) / 2)


      # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2

      # Obter o índice CPI de ambos os anos
      cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
      cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

      # Calcula a média do CPI para ajuste pela inflação
      cpi_media = (cpi_ano1 + cpi_ano2) / 2

      # Ajusta o gasto usando a média do CPI
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


## Função Formatação

In [7]:
# Função para formatar valores em milhões, bilhões ou trilhões para Moeda Local
def formatar_valor_moeda_local(valor, codigo_pais):
    currency_code = get_currency_code(codigo_pais)  # Obtém o código da moeda do país
    if not currency_code:
        currency_code = "Moeda desconhecida"  # Se não houver código, exibe "Moeda desconhecida"

    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{currency_code} {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{currency_code} {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{currency_code} {valor / 1_000_000:.2f} milhões"
    else:
        return f"{currency_code} {valor:.2f}"

# Função para formatar valores em milhões, bilhões ou trilhões para Dólar Ajustado
def formatar_valor_dolar(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"USD {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"USD {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"USD {valor / 1_000_000:.2f} milhões"
    else:
        return f"USD {valor:.2f}"

## Conversor

In [8]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar as novas funções para as colunas 'Gasto_Local' e 'Gasto_Convertido_Dolar_2024'
df['Gasto Local'] = df.apply(lambda row: formatar_valor_moeda_local(row['Gasto_Local'], row['País_Moeda']), axis=1)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Exibir o DataFrame com as colunas formatadas
df[['País do Programa', 'Programa', 'Descrição', 'Ano', 'Ano2', 'Gasto Local', 'Gasto Convertido US$ 2024']]

ValueError: Cannot set a DataFrame with multiple columns to the single column Gasto_Convertido_Dolar_2024